In [1]:
import numpy as np
from pytorch3d.transforms import matrix_to_rotation_6d
from model.rotation2xyz import Rotation2xyz


def xyz_to_rotation_matrix(xyz):
    x, y, z = xyz

    # Perform necessary calculations to obtain the rotation matrix
    # Replace the following lines with your actual calculation
    cos_x = np.cos(x)
    sin_x = np.sin(x)
    cos_y = np.cos(y)
    sin_y = np.sin(y)
    cos_z = np.cos(z)
    sin_z = np.sin(z)

    # Define the rotation matrix
    rotation_matrix = np.array([[cos_y * cos_z, cos_x * sin_z + sin_x * sin_y * cos_z, sin_x * sin_z - cos_x * sin_y * cos_z],
                                [-cos_y * sin_z, cos_x * cos_z - sin_x * sin_y * sin_z, sin_x * cos_z + cos_x * sin_y * sin_z],
                                [sin_y, -sin_x * cos_y, cos_x * cos_y]])

    return rotation_matrix


def motion_xyz2rot(motion):
    N, J, D = motion.shape
    motion_rot = np.zeros((N, J, 9))
    for i in range(N):
        for j in range(J):
            motion_rot[i, j] = xyz_to_rotation_matrix(motion[i, j])
    return motion_rot


/home/ctq566/miniconda3/envs/mdm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
motion = np.load('/home/ctq566/motion-diffusion-model/save/p2p_quaternion_2_random/001/sample00_rep00.npy')
motion.shape


(120, 22, 3)

In [3]:
motion_xyz2rot(motion).shape

ValueError: could not broadcast input array from shape (3,3) into shape (9,)

In [10]:
rot2xyz = Rotation2xyz(device='cpu', dataset='amass')


In [4]:
import numpy as np


def compute_rotation_matrix(joint_pos, point_pos):
    # Define the original vector
    origin_vector = np.array([0, 0, 1])
    # Define the target vector
    target_vector = point_pos - joint_pos
    # Normalize the target vector
    target_vector = target_vector / np.linalg.norm(target_vector)
    # Compute the cross product of the original and target vectors
    v = np.cross(origin_vector, target_vector)
    # Compute the dot product of the original and target vectors
    c = np.dot(origin_vector, target_vector)
    # Compute the skew symmetric cross-product matrix of v
    v_x = np.array([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
    # Compute the rotation matrix
    rotation_matrix = np.eye(3) + v_x + np.dot(v_x, v_x) * ((1 - c) / (np.linalg.norm(v) ** 2))
    return rotation_matrix


# Test with some data
joint_pos = np.array([0, 0, 0], dtype=float)
point_pos = np.array([1, 1, 1], dtype=float)
rotation_matrix = compute_rotation_matrix(joint_pos, point_pos)
print(rotation_matrix)


# Assuming ‘data’ is your array of shape (120, 24, 3)
# data = np.load(‘/Users/zhangbin/Desktop/motion_data/sample00_rep00.npy’)


[[ 0.78867513 -0.21132487  0.57735027]
 [-0.21132487  0.78867513  0.57735027]
 [-0.57735027 -0.57735027  0.57735027]]


In [14]:
data = motion
rotation_matrices = []
# For each frame...
for i in range(data.shape[0]):
    rotation_matrices_frame = []
    # For the first joint, append an identity matrix
    rotation_matrices_frame.append(np.eye(3))
    # For each pair of joints...
    for j in range(1, data.shape[1]):
        joint_pos = data[i, j-1, :]
        point_pos = data[i, j, :]
        rotation_matrix = compute_rotation_matrix(joint_pos, point_pos)
        rotation_matrices_frame.append(rotation_matrix)
    rotation_matrices.append(rotation_matrices_frame)
# Convert to numpy array
rotation_matrices = np.array(rotation_matrices)

In [15]:
rotation_matrices.shape

(120, 22, 3, 3)

In [25]:
import torch


rot2xyz(x=torch.tensor(rotation_matrices).unsqueeze(0), mask=None, pose_rep='rotmat', glob=True, translation=False, jointstype='smpl', vertstrans=True, betas=None, beta=0, glob_rot=None, get_rotations_back=False)

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 5 is not equal to len(dims) = 4

In [24]:
torch.tensor(rotation_matrices).unsqueeze(0).shape

torch.Size([1, 120, 22, 3, 3])